## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [70]:
import os 
from dotenv import load_dotenv
load_dotenv()  ## loading all the environments variable 

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [71]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "openai/gpt-oss-20b",groq_api_key = groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000220EF8E7430>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000220EF8E4EB0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [72]:
from langchain_core.messages import HumanMessage,SystemMessage
model.invoke([HumanMessage(content="Hi , my name is Hisham and i am a jonur Ai engineer")])

AIMessage(content='Hello Hisham! 👋 It’s great to meet an AI engineer. How can I help you today? Whether you’re looking for coding tips, AI insights, or just want to chat about the latest in tech, I’m here for you.', additional_kwargs={'reasoning_content': 'The user says "Hi , my name is Hisham and i am a jonur Ai engineer". They are introducing themselves. We need to respond appropriately. The user likely wants a friendly response. No special instructions. We should respond politely, maybe ask how we can help. The user says "jonur Ai engineer" maybe they mean "JONUR AI engineer"? It\'s a bit unclear. But we can respond: "Hello Hisham, nice to meet you! How can I assist you today?"'}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 87, 'total_tokens': 251, 'completion_time': 0.173862472, 'completion_tokens_details': {'reasoning_tokens': 104}, 'prompt_time': 0.006912548, 'prompt_tokens_details': None, 'queue_time': 0.045576081, 'total_time': 0.18077502}, 'mo

In [73]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hi my name is Hisham and iam a senior Ai engineer ."),
        AIMessage(content="Hi Hisham! 👋 It’s great to meet a Jonur AI engineer. How can I help you today?', additional_kwargs={'reasoning_content': 'User says: Hi , my name is Hisham and i am a jonur Ai engineer. They likely want a response. There\'s no request. Just greet. There\'s no policy violation. We can respond politely, maybe ask how we can help.")
        , HumanMessage(content="Hey what's my name and what do I do?")
    ]
).content

'You’re Hisham, and you’re a senior AI engineer. If there’s anything else you’d like to dive into—whether it’s a project, a question about AI, or just a chat—just let me know!'

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [74]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [75]:
config={"configurable":{"session_id":"chat1"}}

In [76]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Hisham and I am a Chief AI Engineer")],
    config=config
)

In [77]:
response.content

'Hello Hisham! 👋 It’s great to meet a Chief AI Engineer. How can I assist you today? Are you looking for insights on AI trends, best practices in engineering, or something else? Let me know what you have in mind!'

In [78]:
response=with_message_history.invoke(
    [HumanMessage(content="what's my name ? and my jop is ? ")],
    config=config
)

In [79]:
response.content

'You’re Hisham, and you’re a Chief AI Engineer.'

In [80]:
## change the config --> session Id 
config1 = {"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="what's my name ?")],
    config = config1

)
response.content

'I’m sorry, but I don’t have that information. Could you let me know what you’d like me to call you?'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [81]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder 

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Yor are a helpful assistant . Answer all the question to the nest of your ability ."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [82]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is hisham")]}).content

'Hello Hisham! 👋 How can I help you today?'

In [83]:
with_message_history = RunnableWithMessageHistory(chain , get_session_history)


In [84]:
## change the config --> session Id 
config2 = {"configurable":{"session_id":"chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is hisham")],
    config = config1

)
response.content

'Nice to meet you, Hisham! How can I help you today?'

In [85]:
## change the config --> session Id 
config2 = {"configurable":{"session_id":"chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="what's my name ? ")],
    config = config1

)
response.content

'Your name is Hisham.'

In [86]:
### Add more complexity 

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant . Answer all questions to the best of your ability in {language}: and use emoji"
        
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

In [87]:
response= chain.invoke({"messages":[HumanMessage(content="Hi my name is hisham .")] , "language":"Arabic"})
response.content

'مرحبًا هشام! 😊 كيف يمكنني مساعدتك اليوم؟ 🚀'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [88]:
with_message_history= RunnableWithMessageHistory(chain,get_session_history,
                                                  input_messages_key="messages")

In [89]:
config = {"configurable":{"session_id":"chat5"}}

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi , Iam hisham ")] ,"language":"Arabic"},
    config = config
)
response.content

'مرحباً هشام! 😊 كيف يمكنني مساعدتك اليوم؟ 🚀'

In [90]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="what's my name ? ")] ,"language":"Arabic"},
    config = config
)
response.content

'أنت هِشام! 😊✨'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [97]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [98]:
from operator import itemgetter 

from langchain_core.runnables import RunnablePassthrough

chain =(RunnablePassthrough.assign(messages= itemgetter("messages")|trimmer)
        |prompt
        |model
        )


response = chain.invoke(

    {
        "messages":messages + [HumanMessage(content= "what ice cream do i like ? ")],
        "language":"English"
    }
)
response.content

'You like vanilla ice cream! 🍦'

In [99]:
response = chain.invoke(

    {
        "messages":messages + [HumanMessage(content= "what is math problem i ask ?")],
        "language":"English"
    }
)
response.content

'You asked: **“What is 2\u202f+\u202f2?”** 🤔'

In [100]:
## let's wrap this in the messages History 
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [101]:

config = {"configurable":{"session_id":"chat6"}}



In [102]:
result = with_message_history.invoke(
    {
        "messages":messages + [HumanMessage(content="what's my name?")],
        "language":"Arabic",
        
    },
    config = config,
)

result.content

'اسمي بوب 😊'